# Concentration of banking through acquistions & mergers: network analysis




<u>Concentration of banking</u> is the tendency of capital centralizing more and more around a few financial institutions, creating a financial network where a small number of banks and industrial monopolies wield anormous power over production (or under-production), investment (or lack of it), and commerce (or its stultification). It subordinates the separated regional economies of a society into a singularity.

In this Notebook, we look at this economic property by the trends in acquisitions and mergers by the biggest four commercial banks in the U.S. (here referred to as the Big Four) from the 1960's up until today. The The data also includes branch and subsidiary consolidation throughout time. 


<u>Data source:</u> "Relationships.csv". National Information Center. Federal Financial Instututions Examination Council. Feb. 5, 2024. [Link](https://www.ffiec.gov/npw/FinancialReport/DataDownload).

<u>Data dictionary:</u> "Bulk Data Download Data Dictionary and References Guide." Version 2.0. National Information Center. Federal Financial Instututions Examination Council. Sept., 2023. [Link](https://www.ffiec.gov/npw/StaticData/DataDownload/NPW%20Data%20Dictionary.pdf).








## Table of Contents
1. [Data Cleaning](#0)<br>
    a. [Adding missing legal names](#1)<br>
    b. [Changing the accouting method and transformations codes](#2)<br>
2. [Data Wrangling](#3)<br>
    a. [Chase dataframe development](#4)<br>
    b. [Bank of America dataframe development](#5)<br>
    c. [Wells Fargo dataframe development](#6)<br>
    d. [Citibank dataframe development](#7)<br>
3. [M&A Network Plot Development](#8) <br>
    a. [Network plot for Chase](#9)<br>
    b. [Network plot for Bank of America](#10)<br>
    c. [Network plot for Wells Fargo](#11)<br>
    d. [Network plot for Citibank](#12)<br>



In [2]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import matplotlib
import plotly.graph_objs as go
import plotly
import gunicorn
import dash
import networkx as nx
import matplotlib.pyplot as plt
import Bio
import networkx as nx
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import jupyter_dash
import dash_html_components as html
import dash_mantine_components as dmc


C:\Users\16193\AppData\Local\Temp\ipykernel_19860\3558206401.py:17: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


<a id='Data-Cleaning'></a>
## 1. Data Cleaning <a id="0"></a>

Let's upload the raw data from the National Information Center and convert and denomarlize data so as to have one database from which we can engage in our exploratory data analysis.

The data that contains the information on the mergers and acquisitions of banks is the 'transformations' file. It can be found in the webpage of the Federal Financial Institutions Examination Council: https://www.ffiec.gov/npw/FinancialReport/DataDownload



In [3]:

#Defining the directory path towards the excel file containing the data
excel_dir_trans=r"file\path\transformations.xlsx"

#Load excel file into dataframe
transformations_df=pd.read_excel(excel_dir_trans, sheet_name="transformations_big_7")
transformations_df.head()

,ID_RSSD_PREDECESSOR,ID_RSSD_SUCCESSOR,D_DT_TRANS,TRNSFM_CD,ACCT_METHOD,DT_TRANS,Year
0,477639,35785,2024-01-17,1,0,20240117,2024
1,900379,864471,2024-01-03,1,2,20240103,2024
2,3837430,5726556,2024-01-03,1,0,20240103,2024
3,3837467,5726556,2024-01-03,1,0,20240103,2024
4,30650,3971,2024-01-01,1,1,20240101,2024


In [ ]:
# Define the RSSD IDs of the Big Seven banks
big_seven_rssd = {
    'JPMORGAN CHASE BK NA/JPMORGAN CHASE & CO': 852218,
    'BANK OF AMER NA/BANK OF AMER CORP': 480228,
    'CITIBANK NA/CITIGROUP': 476810,
    'WELLS FARGO BK NA/WELLS FARGO & CO': 451965,
    'U S BK NA/U S BC': 504713,
    'TRUIST BK/TRUIST FC': 852320,
    'PNC BK NA/PNC FNCL SVC GROUP': 817824
}

### 1a) Adding missing legal names <a id="1"></a>
The first problem that we encounter with the raw data is that we don't have the legal names of the banks. For this reason, we have to relly on the the 'attributes' dataset of active and closed institutions in the same webpage. This datasets have the RSSD and the legal name through which we can correlate the RSSD of our 'transformations' datataset. 

In [4]:
#Defining the attributes_active dataframe

#Defining the directory path towards the excel file containing the data
excel_dir_attr_act=r"C:file\path\transformations.xlsx"

#Load excel file into dataframe
attr_act_df=pd.read_excel(excel_dir_attr_act, sheet_name="attributes_active")

attr_act_df.head()


,ID_RSSD,D_DT_START,D_DT_END,BHC_IND,BROAD_REG_CD,CHTR_AUTH_CD,CHTR_TYPE_CD,FBO_4C9_IND,FHC_IND,FUNC_REG,...,URL,SLHC_IND,SLHC_TYPE_IND,PRIM_FED_REG,STATE_INC_CD,CNTRY_INC_CD,STATE_INC_ABBR_NM,CNTRY_INC_NM,ID_LEI,IHC_IND
0,56294,2008-12-31,9999-12-31 00:00:00,0,2,2,330,0,0,0,...,0,0,0,NCUA,0,0,0,NaN,0,0
1,56351,2019-01-22,9999-12-31 00:00:00,0,1,1,200,0,0,0,...,WWW.CNBTEXAS.COM,0,0,OCC,0,0,0,NaN,0,0
2,56548,2023-04-03,9999-12-31 00:00:00,0,1,2,200,0,0,0,...,0,0,0,FDIC,55,0,WI,NaN,549300LC0BP2X1F6BE13,0
3,56678,2012-05-17,9999-12-31 00:00:00,0,2,1,330,0,0,0,...,0,0,0,NCUA,0,0,0,NaN,0,0
4,56717,2017-10-26,9999-12-31 00:00:00,0,1,2,200,0,0,0,...,0,0,0,FDIC,42,0,PA,NaN,549300JSQZG5CD2LWR30,0


In [6]:
#Defining the attributes_closed dataframe

#Defining the directory path towards the excel file containing the data
excel_dir_attr_cls=r"file\path\transformations.xlsx"

#Load excel file into dataframe
attr_cls_df=pd.read_excel(excel_dir_attr_cls, sheet_name="attributes_closed")

#Display the dataframe
attr_cls_df.head()

,ID_RSSD,D_DT_START,D_DT_END,BHC_IND,BROAD_REG_CD,CHTR_AUTH_CD,CHTR_TYPE_CD,FBO_4C9_IND,FHC_IND,FUNC_REG,...,URL,SLHC_IND,SLHC_TYPE_IND,PRIM_FED_REG,STATE_INC_CD,CNTRY_INC_CD,STATE_INC_ABBR_NM,CNTRY_INC_NM,ID_LEI,IHC_IND
0,56285,1979-12-31 00:00:00,1982-11-16,0,2,1,330,0,0,0,...,0,0,0,NCUA,0,0,0,NaN,0,0
1,56379,1989-08-09 00:00:00,1993-05-28,0,2,1,300,0,0,0,...,0,0,0,OTS,0,0,0,NaN,0,0
2,56388,1979-12-31 00:00:00,1980-03-21,0,2,1,330,0,0,0,...,0,0,0,NCUA,0,0,0,NaN,0,0
3,56397,1980-12-31 00:00:00,1984-10-03,0,2,2,330,0,0,0,...,0,0,0,NCUA,0,0,0,NaN,0,0
4,56472,1986-01-01 00:00:00,1986-05-04,0,2,1,300,0,0,0,...,0,0,0,NaN,0,0,0,NaN,0,0


Now, let's combine the active and closed 'attributes' datasets to create a list of RSSD's and their respective names. After this, we will map the legal names to the RSSD of the 'transformations' dataset into the 'trasformations_df'.

In [11]:
#Combine the active and closed attributes dataframes
attr_df=pd.concat([attr_act_df, attr_cls_df], axis=0)

#Download the dataframe as a excel file
attr_df.to_excel(r"file\path\transformations.xlsx")

In [17]:
#Create a dictionary to map ID_RSSD to NM_LGL
rssd_nm_dict=pd.Series(attr_df.NM_LGL.values,
                       index=attr_df.ID_RSSD)
rssd_nm_dict.head()


ID_RSSD
56294    SPRINGFIELD FIREFIGHTERS CREDIT UNION         ...
56351    VERABANK, NATIONAL ASSOCIATION                ...
56548    DMB COMMUNITY BANK                            ...
56678    FRANKLIN TRUST FEDERAL CREDIT UNION           ...
56717    DIME BANK, THE                                ...
dtype: object

In [18]:
#Map the legal names to the transformations dataframe
transformations_df['Predecessor_Name']=transformations_df['ID_RSSD_PREDECESSOR'].map(rssd_nm_dict)
transformations_df['Successor_Name']=transformations_df['ID_RSSD_SUCCESSOR'].map(rssd_nm_dict)
transformations_df.head()

,ID_RSSD_PREDECESSOR,ID_RSSD_SUCCESSOR,D_DT_TRANS,TRNSFM_CD,ACCT_METHOD,DT_TRANS,Year,Predecessor_Name,Successor_Name
0,477639,35785,2024-01-17,1,0,20240117,2024,SUNSOUTH BANK ...,ALL IN FEDERAL CREDIT UNION ...
1,900379,864471,2024-01-03,1,2,20240103,2024,NaN,PIEDMONT FEDERAL SAVINGS BANK ...
2,3837430,5726556,2024-01-03,1,0,20240103,2024,"WAKE FOREST BANCSHARES, INC. ...",PIEDMONT FINANCIAL HOLDING COMPANY ...
3,3837467,5726556,2024-01-03,1,0,20240103,2024,"WAKE FOREST BANCORP, M.H.C. ...",PIEDMONT FINANCIAL HOLDING COMPANY ...
4,30650,3971,2024-01-01,1,1,20240101,2024,NaN,MIDCOUNTRY BANK ...


### 1b) Changing the accouting method and transformation codes <a id="2"></a>

In [75]:
# Change the header of 'TRNSFM_CD' to 'Transformation Code'
transformations_df = transformations_df.rename(columns={'TRNSFM_CD': 'Transformation Code'})

# Change the codes in the 'Transformation Code' column
transformations_df['Transformation Code'] = transformations_df['Transformation Code'].replace({
    1: 'Charter Discontinued',
    5: 'Split',
    7: 'Sale of Assets',
    9: 'Charter Retained',
    50: 'Failure'
})


In [76]:
# Change the header of 'ACCT_METHOD' to 'Accounting Method'
transformations_df = transformations_df.rename(columns={'ACCT_METHOD': 'Accounting Method'})

# Change the codes in the 'Accounting Method' column
transformations_df['Accounting Method'] = transformations_df['Accounting Method'].replace({
    0: 'Not Applicable',
    1: 'Merger',
    2: 'Purchase/Acquisition'
})

In [79]:
#Displayed changed transformations_df dataframe with the new headers and codes
transformations_df.head()

,ID_RSSD_PREDECESSOR,ID_RSSD_SUCCESSOR,D_DT_TRANS,Transformation Code,Accounting Method,DT_TRANS,Year,Predecessor_Name,Successor_Name
0,477639,35785,2024-01-17,Charter Discontinued,Not Applicable,20240117,2024,SUNSOUTH BANK ...,ALL IN FEDERAL CREDIT UNION ...
1,900379,864471,2024-01-03,Charter Discontinued,Purchase/Acquisition,20240103,2024,NaN,PIEDMONT FEDERAL SAVINGS BANK ...
2,3837430,5726556,2024-01-03,Charter Discontinued,Not Applicable,20240103,2024,"WAKE FOREST BANCSHARES, INC. ...",PIEDMONT FINANCIAL HOLDING COMPANY ...
3,3837467,5726556,2024-01-03,Charter Discontinued,Not Applicable,20240103,2024,"WAKE FOREST BANCORP, M.H.C. ...",PIEDMONT FINANCIAL HOLDING COMPANY ...
4,30650,3971,2024-01-01,Charter Discontinued,Merger,20240101,2024,NaN,MIDCOUNTRY BANK ...


In [ ]:
#Download the transformations_df as a excel file
transformations_df.to_excel(r"file\path\transformations.xlsx")

Although we a feasible dataframe in 'transformation_df', we don't have a complete list of the legal names of the predecessor entities. This is mainly because the 'attributes' datasets do not register branch divisions as seperate entities from the national association bank. Therefore, we will need to download the 'transformations_df' dataframe into an excel file 'trans_df' and fill the missing data entries manually for the banks of interest. 

In [2]:
#Create a data from from the 'trans_df' excel file
file_path=r"file\path\transformations.xlsx"

In [42]:
#Load excel file 'trans_df.xlsx'into dataframe
trans_df=pd.read_excel(file_path, sheet_name="Sheet1")
trans_df.head()

,Column1,ID_RSSD_PREDECESSOR,ID_RSSD_SUCCESSOR,D_DT_TRANS,Transformation Code,Accounting Method,DT_TRANS,Year,Predecessor_Name,Successor_Name
0,0,477639,35785,2024-01-17,Charter Discontinued,Not Applicable,20240117,2024,SUNSOUTH BANK ...,ALL IN FEDERAL CREDIT UNION ...
1,1,900379,864471,2024-01-03,Charter Discontinued,Purchase/Acquisition,20240103,2024,NaN,PIEDMONT FEDERAL SAVINGS BANK ...
2,2,3837430,5726556,2024-01-03,Charter Discontinued,Not Applicable,20240103,2024,"WAKE FOREST BANCSHARES, INC. ...",PIEDMONT FINANCIAL HOLDING COMPANY ...
3,3,3837467,5726556,2024-01-03,Charter Discontinued,Not Applicable,20240103,2024,"WAKE FOREST BANCORP, M.H.C. ...",PIEDMONT FINANCIAL HOLDING COMPANY ...
4,4,30650,3971,2024-01-01,Charter Discontinued,Merger,20240101,2024,NaN,MIDCOUNTRY BANK ...


## 2) Data Wrangling <a id="3"></a>
Now that we have standardized our data, we need to create seperate dataframes for each of the Big Four banks to add flexibility to our network mapping. This is because, despite combining our dataframes of the 'properties' and 'transformations' datasets allowed us to find names of most of the transformation instances, branch consolidated transformations were not named with the legal name. This is perhaps because the 'properties' dataset does not recognize branch entities as seperate entities from national association. 


As a result of this, we will be adding this names manually by searching for the RSSD of the predecessor entity in the [National Information Center RSSD search engine](https://www.ffiec.gov/npw). 

Additionally, we are going to create a list of all the branch entities of the national association so we can latter use this dataframe to add a switch bottom on our Dash app to allow the user to filter out all those data points related to brach consolidation.

### 2a) Chase dataframe development <a id="4"></a>

In [5]:
# Filter the 'transformations_df' dataframe based on the ID_RSSD_SUCCESSOR
trans_chase_df = trans_df[trans_df['ID_RSSD_SUCCESSOR'] == 852218]


In [6]:
#Rename the successor to 'Chase'
trans_chase_df ['Successor_Name']='Chase'

C:\Users\16193\AppData\Local\Temp\ipykernel_3080\3583578456.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans_chase_df ['Successor_Name']='Chase'


In [45]:
#Display the dataframe for the Chase bank
trans_chase_df.head()

,Column1,ID_RSSD_PREDECESSOR,ID_RSSD_SUCCESSOR,D_DT_TRANS,Transformation Code,Accounting Method,DT_TRANS,Year,Predecessor_Name,Successor_Name
0,318,4114567,852218,2023-05-01,Failure,Not Applicable,20230501,2023,FIRST REPUBLIC BANK ...,Chase
1,1108,3179318,852218,2021-12-31,Charter Discontinued,Merger,20211231,2021,ONE MORTGAGE PARTNERS CORP. ...,Chase
2,2372,3118474,852218,2019-10-20,Charter Discontinued,Merger,20191020,2019,J.P. MORGAN TREASURY TECHNOLOGIES CORPORATION ...,Chase
3,2620,489913,852218,2019-05-19,Charter Discontinued,Merger,20190519,2019,"CHASE BANK USA, NATIONAL ASSOCIATION ...",Chase
4,2621,1451364,852218,2019-05-19,Charter Discontinued,Merger,20190519,2019,"CHASE BANKCARD SERVICES, INC. ...",Chase


In [110]:
#Now let's download the 'trans_chase_df' as an excel file for revision
path_to_tables = r"file\path\transformations.xlsx"
trans_chase_df.to_excel(path_to_tables + '\\trans_chase_df.xlsx', index=False)


Create a list of all the legal names of all the branches that were consolidated under Chase. 

In [29]:
#Upload the dataframe for Chase that juxtaposes the column of predecessors  that has the branch legal names and the column that does not
path_to_branch_tables=r"file\path\transformations.xlsx"
chase_branch_df=pd.read_excel(path_to_branch_tables + '\\chase_branch_con.xlsx')
chase_branch_df.head()

,ID_RSSD_PREDECESSOR,ID_RSSD_SUCCESSOR,D_DT_TRANS,Transformation Code,Accounting Method,DT_TRANS,Year,Predecessor_Name,Predecessor_Name_Branch,Successor_Name
0,4114567,852218,2023-05-01,Failure,Not Applicable,20230501,2023,FIRST REPUBLIC BANK ...,FIRST REPUBLIC BANK ...,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION ..."
1,3179318,852218,2021-12-31,Charter Discontinued,Merger,20211231,2021,ONE MORTGAGE PARTNERS CORP. ...,ONE MORTGAGE PARTNERS CORP. ...,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION ..."
2,3118474,852218,2019-10-20,Charter Discontinued,Merger,20191020,2019,J.P. MORGAN TREASURY TECHNOLOGIES CORPORATION ...,J.P. MORGAN TREASURY TECHNOLOGIES CORPORATION ...,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION ..."
3,489913,852218,2019-05-19,Charter Discontinued,Merger,20190519,2019,"CHASE BANK USA, NATIONAL ASSOCIATION ...","CHASE BANK USA, NATIONAL ASSOCIATION ...","JPMORGAN CHASE BANK, NATIONAL ASSOCIATION ..."
4,1451364,852218,2019-05-19,Charter Discontinued,Merger,20190519,2019,"CHASE BANKCARD SERVICES, INC. ...","CHASE BANKCARD SERVICES, INC. ...","JPMORGAN CHASE BANK, NATIONAL ASSOCIATION ..."


In [31]:
#Create a list of the missing points
chase_missing_points=list(set(chase_branch_df['Predecessor_Name_Branch'])-set(chase_branch_df['Predecessor_Name']))
chase_missing_points

['BINGHAMTON BRANCH [CHASE MANHATTAN BANK, THE]',
 'GREENWICH OFFICE [GLENS FALLS NATIONAL BANK AND TRUST COMPANY]',
 'HILTON BRANCH [MANUFACTURERS AND TRADERS TRUST COMPANY]',
 '225 SOUTH STREET BRANCH [JPMORGAN CHASE BANK, NATIONAL ASSOCIATION]',
 'HEMPSTEAD BRANCH [JPMORGAN CHASE BANK]',
 'BROAD STREET DOWNTOWN [JPMORGAN CHASE BANK, NATIONAL ASSOCIATION]',
 'NYACK BRANCH [MANUFACTURERS AND TRADERS TRUST COMPANY]',
 'CHICAGO MAIN [JPMORGAN CHASE BANK, NATIONAL ASSOCIATION]',
 '60 WALL STREET BRANCH [JPMORGAN CHASE BANK, NATIONAL ASSOCIATION]',
 'PLUM STREET BRANCH [CHASE MANHATTAN BANK, THE]',
 'SMITHTOWN BRANCH [JPMORGAN CHASE BANK, NATIONAL ASSOCIATION]',
 'KIMBALL JUNCTION BRANCH [JPMORGAN CHASE BANK, NATIONAL ASSOCIATION]',
 'CHASE PLAZA [JPMORGAN CHASE BANK, NATIONAL ASSOCIATION]',
 'HOUSTON MAIN BRANCH [JPMORGAN CHASE BANK, NATIONAL ASSOCIATION]',
 '277 PARK AVENUE BRANCH [CHEMICAL BANK]',
 'BUFFALO BRANCH [MANUFACTURERS AND TRADERS TRUST COMPANY]']

In [ ]:
# Define the list of branch consolidation entities for the switch filter

chase_branch_list=[
 'BINGHAMTON BRANCH [CHASE MANHATTAN BANK, THE]',
 'GREENWICH OFFICE [GLENS FALLS NATIONAL BANK AND TRUST COMPANY]',
 'HILTON BRANCH [MANUFACTURERS AND TRADERS TRUST COMPANY]',
 '225 SOUTH STREET BRANCH [JPMORGAN CHASE BANK, NATIONAL ASSOCIATION]',
 'HEMPSTEAD BRANCH [JPMORGAN CHASE BANK]',
 'BROAD STREET DOWNTOWN [JPMORGAN CHASE BANK, NATIONAL ASSOCIATION]',
 'NYACK BRANCH [MANUFACTURERS AND TRADERS TRUST COMPANY]',
 'CHICAGO MAIN [JPMORGAN CHASE BANK, NATIONAL ASSOCIATION]',
 '60 WALL STREET BRANCH [JPMORGAN CHASE BANK, NATIONAL ASSOCIATION]',
 'PLUM STREET BRANCH [CHASE MANHATTAN BANK, THE]',
 'SMITHTOWN BRANCH [JPMORGAN CHASE BANK, NATIONAL ASSOCIATION]',
 'KIMBALL JUNCTION BRANCH [JPMORGAN CHASE BANK, NATIONAL ASSOCIATION]',
 'CHASE PLAZA [JPMORGAN CHASE BANK, NATIONAL ASSOCIATION]',
 'HOUSTON MAIN BRANCH [JPMORGAN CHASE BANK, NATIONAL ASSOCIATION]',
 '277 PARK AVENUE BRANCH [CHEMICAL BANK]',
 'BUFFALO BRANCH [MANUFACTURERS AND TRADERS TRUST COMPANY]'
]

### 2b) Bank of American dataframe development <a id="5"></a>

In [56]:
#Filter the 'transformation_df' dataframe based on the ID_RSSD of Bank of America
trans_boa_df=trans_df[trans_df['ID_RSSD_SUCCESSOR']==480228]

#Rename the successor to 'Bank of America'
trans_boa_df['Successor_Name']='Bank of America'

#Display the dataframe for the Bank of America
trans_boa_df.head()

C:\Users\16193\AppData\Local\Temp\ipykernel_3080\1291295719.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Column1,ID_RSSD_PREDECESSOR,ID_RSSD_SUCCESSOR,D_DT_TRANS,Transformation Code,Accounting Method,DT_TRANS,Year,Predecessor_Name,Successor_Name
2043,2043,3313943,480228,2020-03-31,Charter Discontinued,Merger,20200331,2020,"TAPO CANYON BRANCH [BANK OF AMERICA, NATIONAL ...",Bank of America
6997,6997,1830035,480228,2014-10-01,Charter Discontinued,Merger,20141001,2014,"FIA CARD SERVICES, NATIONAL ASSOCIATION ...",Bank of America
8458,8458,2867056,480228,2013-04-01,Charter Discontinued,Merger,20130401,2013,"BANK OF AMERICA OREGON, NATIONAL ASSOCIATION ...",Bank of America
8462,8462,3344611,480228,2013-04-01,Charter Discontinued,Merger,20130401,2013,"BANK OF AMERICA RHODE ISLAND, NATIONAL ASSOCIA...",Bank of America
10403,10403,2496120,480228,2011-07-01,Charter Discontinued,Merger,20110701,2011,MERRILL LYNCH CREDIT CORPORATION ...,Bank of America


In [57]:
#Download the 'trans_boa_df' as an excel file for revision
path_to_tables = r"file\path\transformations.xlsx"
trans_boa_df.to_excel(path_to_tables + '\\trans_boa_df.xlsx', index=False)

Create a list of all the legal names of all the branches that were consolidated under Bank of America. 

In [26]:
#Upload the dataframe for Bank of America that juxtaposes the column of predecessors  that has the branch legal names and the column that does not
path_to_branch_tables=r"file\path\transformations.xlsx"
boa_branch_df=pd.read_excel(path_to_branch_tables + '\\boa_branch_con.xlsx')
boa_branch_df.head()

,Column1,ID_RSSD_PREDECESSOR,ID_RSSD_SUCCESSOR,D_DT_TRANS,Transformation Code,Accounting Method,DT_TRANS,Year,Predecessor_Name,Predecessor_Name_Branch,Successor_Name
0,2043,3313943,480228,2020-03-31,Charter Discontinued,Merger,20200331,2020,NaN,"TAPO CANYON BRANCH [BANK OF AMERICA, NATIONAL ...","BANK OF AMERICA, NATIONAL ASSOCIATION ..."
1,6997,1830035,480228,2014-10-01,Charter Discontinued,Merger,20141001,2014,"FIA CARD SERVICES, NATIONAL ASSOCIATION ...","FIA CARD SERVICES, NATIONAL ASSOCIATION ...","BANK OF AMERICA, NATIONAL ASSOCIATION ..."
2,8458,2867056,480228,2013-04-01,Charter Discontinued,Merger,20130401,2013,"BANK OF AMERICA OREGON, NATIONAL ASSOCIATION ...","BANK OF AMERICA OREGON, NATIONAL ASSOCIATION ...","BANK OF AMERICA, NATIONAL ASSOCIATION ..."
3,8462,3344611,480228,2013-04-01,Charter Discontinued,Merger,20130401,2013,"BANK OF AMERICA RHODE ISLAND, NATIONAL ASSOCIA...","BANK OF AMERICA RHODE ISLAND, NATIONAL ASSOCIA...","BANK OF AMERICA, NATIONAL ASSOCIATION ..."
4,10403,2496120,480228,2011-07-01,Charter Discontinued,Merger,20110701,2011,MERRILL LYNCH CREDIT CORPORATION ...,MERRILL LYNCH CREDIT CORPORATION ...,"BANK OF AMERICA, NATIONAL ASSOCIATION ..."


In [27]:
#Create a list of all the missing points in the dataframe
boa_missing_points = list(set(boa_branch_df['Predecessor_Name_Branch']) - set(boa_branch_df['Predecessor_Name']))
boa_missing_points

['COLUMBIA DOWNTOWN BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'FORT DODGE BRANCH [NORTHWEST BANK]',
 'DOWNTOWN JACKSONVILLE BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'ASHEVILLE BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'NEW YORK BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'QUINCY BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'PINE BLUFF MAIN BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'NORTH AKARD STREET BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'SANIBEL BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'CAMDEN BRANCH [FARMERS BANK & TRUST COMPANY]',
 'HOME OFFICE [RBC CENTURA BANK]',
 'BATESVILLE MAIN BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'HILLSBORO MAIN BRANCH [FIRST COMMUNITY BANK OF HILLSBORO]',
 'FAYETTEVILLE EAST CENTER BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'JACKSONVILLE BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'MOUNT VERNON MAIN BRANCH [FIRST FINANCIAL BANK, NATIONAL ASSOCIATION]',
 'BANK OF AMERICA

In [28]:
# Define the list of branch consolidation entities for the switch filter
boa_branch_list = [
 'COLUMBIA DOWNTOWN BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'FORT DODGE BRANCH [NORTHWEST BANK]',
 'DOWNTOWN JACKSONVILLE BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'ASHEVILLE BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'NEW YORK BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'QUINCY BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'PINE BLUFF MAIN BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'NORTH AKARD STREET BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'SANIBEL BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'CAMDEN BRANCH [FARMERS BANK & TRUST COMPANY]',
 'HOME OFFICE [RBC CENTURA BANK]',
 'BATESVILLE MAIN BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'HILLSBORO MAIN BRANCH [FIRST COMMUNITY BANK OF HILLSBORO]',
 'FAYETTEVILLE EAST CENTER BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'JACKSONVILLE BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'MOUNT VERNON MAIN BRANCH [FIRST FINANCIAL BANK, NATIONAL ASSOCIATION]',
 'BANK OF AMERICA CENTER TULSA BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'RICHMOND CENTER BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'LINCOLNTON BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'FIELD BUILDING BRANCH, THE [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'MARSHALL MAIN BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'BENTON BRANCH PUBLIC SQUARE [FIRST FINANCIAL BANK, NATIONAL ASSOCIATION]',
 'SOUTHWEST MISSOURI BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 '114 WEST 47TH STREET BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'SKYWALK BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'PWM LITTLE ROCK BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'ROCK HILL BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'ROSWELL MAIN [WASHINGTON FEDERAL, NATIONAL ASSOCIATION]',
 'RICHLAND BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'HOT SPRINGS MAIN BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'WEST PLAINS MAIN BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'PWM LASALLE MIDWEST BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'DELAWARE AVENUE BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'RATON BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'TAPO CANYON BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'LAS CRUCES BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'FARMINGTON BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'GLOUCESTER STREET BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'RIO ARRIBA MAIN BRANCH [WASHINGTON FEDERAL BANK]',
 'SILVER CITY MAIN BRANCH [WASHINGTON FEDERAL BANK]',
 'ONE KANSAS CITY PLACE BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'SANTA TERESA NM BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'CALIFORNIA STREET BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'NEWARK FRONT STREET BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'ALBUQUERQUE BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'SPENCER BRANCH [NORTHWEST BANK]',
 'BOATMENS VANDALIA BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'SOUTHERN MISSOURI BR [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'CHERRY CREEK BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'LEBANON BRANCH [ARVEST BANK]',
 'ATLANTA BANK OF AMERICA PLAZA BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'SANTA FE BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'BOONVILLE BRANCH [FIRST STATE COMMUNITY BANK]',
 'RUSSELLVILLE MAIN BRANCH [ARVEST BANK]',
 'JONESBORO MAIN BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'BULL SHOALS MAIN BRANCH [FIRST SECURITY BANK]',
 'HARKRIDER NORTH BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'WILLIAM STREET BRANCH [FIRST STATE COMMUNITY BANK]',
 'MASON CITY BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'MIDTOWN BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'RIVER VALLEY BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'SAINT LOUIS MAIN BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'GALLUP BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'KENNEDY PLAZA [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'NEW BERN OFFICE [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'HOBBS MAIN BRANCH [WASHINGTON FEDERAL BANK]',
 'BANK OF AMERICA PLAZA BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'TROY BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'HIGHLANDS BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'PLAZA SAN JACINTO BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'COLES COUNTY BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'HENDERSON OFFICE [NATIONSBANK OF NORTH CAROLINA, NATIONAL ASSOCIATION]',
 'CLOVIS BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'KENNETT BRANCH [FIRST STATE COMMUNITY BANK]',
 'OSAGE BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'ROLLA MAIN BRANCH [FIRST STATE COMMUNITY BANK]',
 'NASHVILLE MAIN BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'SALT LAKE CITY BRANCH [BANK OF AMERICA, NATIONAL ASSOCIATION]',
 'MOUNT AIRY OFFICE [FIRST COMMUNITY BANK]']


### 2c) Wells Fargo dataframe development <a id="6"></a>

In [59]:
#Filter the 'transformation_df' dataframe based on the ID_RSSD of Wells Fargo
trans_wf_df=trans_df[trans_df['ID_RSSD_SUCCESSOR']==451965]

#Rename the successor to 'Wells Farog'
trans_wf_df['Successor_Name']='Wells Fargo'

#Display the dataframe for the Bank of America
trans_wf_df.head()

C:\Users\16193\AppData\Local\Temp\ipykernel_3080\1479698906.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Column1,ID_RSSD_PREDECESSOR,ID_RSSD_SUCCESSOR,D_DT_TRANS,Transformation Code,Accounting Method,DT_TRANS,Year,Predecessor_Name,Successor_Name
9503,9503,1027143,451965,2012-02-01,Charter Discontinued,Merger,20120201,2012,"WELLS FARGO AUTO FINANCE, LLC ...",Wells Fargo
10341,10341,2575052,451965,2011-07-15,Charter Discontinued,Merger,20110715,2011,"FIRST UNION FINANCIAL INVESTMENTS, LLC ...",Wells Fargo
10344,10344,3536537,451965,2011-07-15,Charter Discontinued,Merger,20110715,2011,WES HOLDING CORPORATION ...,Wells Fargo
10398,10398,1079544,451965,2011-07-01,Charter Discontinued,Merger,20110701,2011,SOUTHTRUST MORTGAGE CORPORATION ...,Wells Fargo
10399,10399,1117660,451965,2011-07-01,Charter Discontinued,Merger,20110701,2011,MERIDIAN MORTGAGE CORPORATION ...,Wells Fargo


In [60]:
#Download the 'trans_wf_df' as an excel file for revision
path_to_tables = r"file\path\transformations.xlsx"
trans_wf_df.to_excel(path_to_tables + '\\trans_wf_df.xlsx', index=False)

Create a list of all the legal names of all the branches that were consolidated under Wells Fargo. 

In [39]:
#Upload the dataframe for Wells Fargo that juxtaposes the column of predecessors  that has the branch legal names and the column that does not
path_to_branch_tables=r"file\path\transformations.xlsx"
wf_branch_df=pd.read_excel(path_to_branch_tables + '\\wf_branch_con.xlsx')
wf_branch_df.head()

,ID_RSSD_PREDECESSOR,ID_RSSD_SUCCESSOR,D_DT_TRANS,Transformation Code,Accounting Method,DT_TRANS,Year,Predecessor_Name,Predecessor_Name_Branch,Successor_Name
0,1027143,451965,2012-02-01,Charter Discontinued,Merger,20120201,2012,"WELLS FARGO AUTO FINANCE, LLC ...","WELLS FARGO AUTO FINANCE, LLC ...","WELLS FARGO BANK, NATIONAL ASSOCIATION ..."
1,2575052,451965,2011-07-15,Charter Discontinued,Merger,20110715,2011,"FIRST UNION FINANCIAL INVESTMENTS, LLC ...","FIRST UNION FINANCIAL INVESTMENTS, LLC ...","WELLS FARGO BANK, NATIONAL ASSOCIATION ..."
2,3536537,451965,2011-07-15,Charter Discontinued,Merger,20110715,2011,WES HOLDING CORPORATION ...,WES HOLDING CORPORATION ...,"WELLS FARGO BANK, NATIONAL ASSOCIATION ..."
3,1079544,451965,2011-07-01,Charter Discontinued,Merger,20110701,2011,SOUTHTRUST MORTGAGE CORPORATION ...,SOUTHTRUST MORTGAGE CORPORATION ...,"WELLS FARGO BANK, NATIONAL ASSOCIATION ..."
4,1117660,451965,2011-07-01,Charter Discontinued,Merger,20110701,2011,MERIDIAN MORTGAGE CORPORATION ...,MERIDIAN MORTGAGE CORPORATION ...,"WELLS FARGO BANK, NATIONAL ASSOCIATION ..."


In [40]:
#Create a list of teh missing points in the dataframe
wf_missing_points=list(set(wf_branch_df['Predecessor_Name_Branch'])-set(wf_branch_df['Predecessor_Name']))
wf_missing_points

['PAJARO VALLEY BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'BOISE MAIN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'FARGO MAIN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'BARRANCA TOWERS BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'SEATTLE MAIN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'SOLANA BEACH BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'BEVERLY HILLS BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'DOWNTOWN MB BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'MID-PENINSULA BANK MAIN, COWPER BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'DRIGGS BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'PINEDALE BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'SHERIDAN MAIN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'WELLS FARGO BANK SAN FRANCISCO OFFICE [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'DENVER MAIN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 '5TH & JEFFERSON BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATIO

In [ ]:
# Define the list of branch consolidation entities for the switch filter
wf_branch_df=[
 'PAJARO VALLEY BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'BOISE MAIN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'FARGO MAIN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'BARRANCA TOWERS BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'SEATTLE MAIN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'SOLANA BEACH BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'BEVERLY HILLS BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'DOWNTOWN MB BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'MID-PENINSULA BANK MAIN, COWPER BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'DRIGGS BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'PINEDALE BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'SHERIDAN MAIN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'WELLS FARGO BANK SAN FRANCISCO OFFICE [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'DENVER MAIN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 '5TH & JEFFERSON BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'WELLS FARGO FINANCIAL BANK BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'CANON MAIN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'NORTHERN LIGHTS & C STREET BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'MESA MALL BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'ANAHEIM BRANCH #3 [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'VENTURA OFFICE [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'GLENDALE BRAND OFFICE [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'PHOENIX MAIN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'SANTA MARIA BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'TOWER BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'GATEWAY BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'DIXIE FARM ROAD STORE BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'VAN WERT BRANCH [FLAGSTAR BANK, NATIONAL ASSOCIATION]',
 'NAPA DOWNTOWN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'TENTH & SHIPLEY BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'CASPER DOWNTOWN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'JACKSON TOWN SQUARE BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'OMAHA MAIN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'FELICITA VILLAGE BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'SANTA CRUZ BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'CENTRAL & WASHINGTON BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'LAS VEGAS TOWER BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'WELLS FARGO CENTER SKYWAY BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'HARBOR BOULEVARD BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'MARQUETTE DOWNTOWN BRANCH [FLAGSTAR BANK, NATIONAL ASSOCIATION]',
 'CODY MAIN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'BILLINGS DOWNTOWN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'ROGERS SOUTH BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'ORANGE PLAZA BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'SANTA FE MAIN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'BUENA PARK BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'SAN FRANCISCO MAIN OFFICE [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'INDIANA CENTER BRANCH [FLAGSTAR BANK, NATIONAL ASSOCIATION]',
 'DOWNTOWN AUBURN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'HOLTVILLE BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'MARKET FINANCIAL BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'MARYSVILLE BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'AZUSA BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'RICHMOND ROAD BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'JACKSON BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'SAN FRANCISCO MAIN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'COLORADO-ORANGE GROVE BLVD BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'MILWAUKEE MAIN BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'NORTH LAS VEGAS BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'THOUSAND OAKS BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'LOMAS BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'PUEBLO MESSENGER SERVICE BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'SAN LUIS OBISPO BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'GALESBURG BRANCH [WELLS FARGO BANK, NATIONAL ASSOCIATION]',
 'PARADISE OFFICE [WELLS FARGO BANK, NATIONAL ASSOCIATION]']


### 2d) Citibank dataframe development <a id="7"></a>


In [61]:
#Filter the 'transformation_df' dataframe based on the ID_RSSD of Citibank
trans_cb_df=trans_df[trans_df['ID_RSSD_SUCCESSOR']==476810]

#Rename the successor to 'Citibank'
trans_cb_df['Successor_Name']='Citibank'

#Display the dataframe for the Citibank
trans_cb_df.head()

C:\Users\16193\AppData\Local\Temp\ipykernel_3080\1108738750.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Column1,ID_RSSD_PREDECESSOR,ID_RSSD_SUCCESSOR,D_DT_TRANS,Transformation Code,Accounting Method,DT_TRANS,Year,Predecessor_Name,Successor_Name
762,762,3382547,476810,2022-07-01,Charter Discontinued,Merger,20220701,2022,DEPARTMENT STORES NATIONAL BANK ...,Citibank
9226,9226,3158498,476810,2012-06-19,Charter Discontinued,Merger,20120619,2012,"CAL FED HOLDINGS, INC. ...",Citibank
9563,9563,374112,476810,2012-01-01,Charter Discontinued,Merger,20120101,2012,"CITICORP TRUST BANK, FSB ...",Citibank
10382,10382,486752,476810,2011-07-01,Charter Discontinued,Merger,20110701,2011,"CITIBANK (SOUTH DAKOTA), NATIONAL ASSOCIATION ...",Citibank
14892,14892,545538,476810,2007-01-01,Charter Discontinued,Merger,20070101,2007,CITIBANK INTERNATIONAL ...,Citibank


In [62]:
#Now let's download the 'trans_cb_df' as an excel file for revision
path_to_tables = r"file\path\transformations.xlsx"
trans_cb_df.to_excel(path_to_tables + '\\trans_cb_df.xlsx', index=False)


Create a list of all the legal names of all the branches that were consolidated under Bank of America. 

In [41]:
#Upload the dataframe for Chase that juxtaposes the column of predecessors  that has the branch legal names and the column that does not
path_to_branch_tables=r"file\path\transformations.xlsx"
cb_branch_df=pd.read_excel(path_to_branch_tables + '\\cb_branch_con.xlsx')
cb_branch_df.head()

,ID_RSSD_PREDECESSOR,ID_RSSD_SUCCESSOR,D_DT_TRANS,Transformation Code,Accounting Method,DT_TRANS,Year,Predecessor_Name,Predecessor_Name_Branch,Successor_Name
0,3382547,476810,2022-07-01,Charter Discontinued,Merger,20220701,2022,DEPARTMENT STORES NATIONAL BANK ...,DEPARTMENT STORES NATIONAL BANK ...,"CITIBANK, N.A. ..."
1,3158498,476810,2012-06-19,Charter Discontinued,Merger,20120619,2012,"CAL FED HOLDINGS, INC. ...","CAL FED HOLDINGS, INC. ...","CITIBANK, N.A. ..."
2,374112,476810,2012-01-01,Charter Discontinued,Merger,20120101,2012,"CITICORP TRUST BANK, FSB ...","CITICORP TRUST BANK, FSB ...","CITIBANK, N.A. ..."
3,486752,476810,2011-07-01,Charter Discontinued,Merger,20110701,2011,"CITIBANK (SOUTH DAKOTA), NATIONAL ASSOCIATION ...","CITIBANK (SOUTH DAKOTA), NATIONAL ASSOCIATION ...","CITIBANK, N.A. ..."
4,545538,476810,2007-01-01,Charter Discontinued,Merger,20070101,2007,CITIBANK INTERNATIONAL ...,CITIBANK INTERNATIONAL ...,"CITIBANK, N.A. ..."


In [42]:
#Create a list of the missing points
cb_missing_points=list(set(cb_branch_df['Predecessor_Name_Branch'])-set(cb_branch_df['Predecessor_Name']))
cb_missing_points

['RESTON BRANCH [CITIBANK, N.A.]',
 'PORT RICHMOND BR [CITIBANK, N.A.]',
 'NEW CASTLE BRANCH [CITIBANK, N.A.]',
 'LA MAIN BRANCH [CITIBANK, N.A.]',
 'CENTRAL VALLEY BRANCH [HUDSON UNITED BANK]',
 '120 BROADWAY BRANCH [CITIBANK, N.A.]',
 'LINCOLN PARK BRANCH [CITIBANK, N.A.]',
 '99 GARNSEY BRANCH [CITIBANK, N.A.]',
 'BAY SHORE BRANCH [CITIBANK, N.A.]',
 'SAN FRANCISCO BRANCH [CITIBANK, N.A.]']

In [ ]:
# Define the list of branch consolidation entities for the switch filter

chase_branch_list=[
    'RESTON BRANCH [CITIBANK, N.A.]',
 'PORT RICHMOND BR [CITIBANK, N.A.]',
 'NEW CASTLE BRANCH [CITIBANK, N.A.]',
 'LA MAIN BRANCH [CITIBANK, N.A.]',
 'CENTRAL VALLEY BRANCH [HUDSON UNITED BANK]',
 '120 BROADWAY BRANCH [CITIBANK, N.A.]',
 'LINCOLN PARK BRANCH [CITIBANK, N.A.]',
 '99 GARNSEY BRANCH [CITIBANK, N.A.]',
 'BAY SHORE BRANCH [CITIBANK, N.A.]',
 'SAN FRANCISCO BRANCH [CITIBANK, N.A.]']


## 3) M&A Network Plot Development <a id="8"></a>

Our nework plot displays all the history of mergers, acquisitions and branch/subisiriary consolidations for each of the Big Four Banks starting in the 1960's until the lastest available data

### 3a) Network plot for Chase <a id="9"></a>

In [64]:
# Create a directed graph
G = nx.DiGraph()

# Define node positions based on the 'Year'
pos = {}
year_min = trans_chase_df['Year'].min()
year_max = trans_chase_df['Year'].max()
y_pos = 0

for _, row in trans_chase_df.iterrows():
    G.add_node(row['Predecessor_Name'], 
               year=row['Year'], 
               trsfm_cd=row['Transformation Code'], 
               acct_method=row['Accounting Method'])
    pos[row['Predecessor_Name']] = (row['Year'], y_pos)
    y_pos += 1

G.add_node('Chase')
pos['Chase'] = (year_max + 1, y_pos / 2)  # Position Chase at the end

# Add edges from predecessors to 'Chase'
for _, row in trans_chase_df.iterrows():
    G.add_edge(row['Predecessor_Name'], row['Successor_Name'])

# Create edge trace for plotly
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

# Create node trace
node_x = [pos[node][0] for node in G.nodes()]
node_y = [pos[node][1] for node in G.nodes()]
node_text = []
for node in G.nodes():
    year = G.nodes[node].get('year', 'N/A')
    trsfm_cd = G.nodes[node].get('trsfm_cd', 'N/A')
    acct_method = G.nodes[node].get('acct_method', 'N/A')
    node_info = f'{node}<br>Year: {year}<br>Transformation Code: {trsfm_cd}<br>Accounting Method: {acct_method}'
    node_text.append(node_info)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    text=node_text,
    marker=dict(
        size=10,
        line_width=2,color='#0D5EAF'))

# Create the figure for plotting
fig_chase = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Network of Chase Acquisitions and Mergers',
                titlefont_size=16,
                showlegend=False,
                hovermode='y unified',
                xaxis=dict(title='Year of Transformation'),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                margin=dict(b=20,l=5,r=5,t=40)) # Set margins to add a title and adjust the plot size. 'b' is the bottom margin, 'l' is the left margin, 'r' is the right margin, and 't' is the top margin.
                )

#Add dark theme
fig_chase.update_layout(template='plotly_dark')

fig_chase.show()

### 3b) Network plot for Bank of America <a id="10"></a>

In [66]:
# Create a directed graph
G = nx.DiGraph()

# Define node positions based on the 'Year'
pos = {}
year_min = trans_boa_df['Year'].min()
year_max = trans_boa_df['Year'].max()
y_pos = 0

for _, row in trans_boa_df.iterrows():
    G.add_node(row['Predecessor_Name'], 
               year=row['Year'], 
               trsfm_cd=row['Transformation Code'], 
               acct_method=row['Accounting Method'])
    pos[row['Predecessor_Name']] = (row['Year'], y_pos)
    y_pos += 1

G.add_node('Bank of America')
pos['Bank of America'] = (year_max + 1, y_pos / 2)  # Position BoA at the end

# Add edges from predecessors to 'Chase'
for _, row in trans_boa_df.iterrows():
    G.add_edge(row['Predecessor_Name'], row['Successor_Name'])

# Create edge trace for plotly
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

# Create node trace
node_x = [pos[node][0] for node in G.nodes()]
node_y = [pos[node][1] for node in G.nodes()]
node_text = []
for node in G.nodes():
    year = G.nodes[node].get('year', 'N/A')
    trsfm_cd = G.nodes[node].get('trsfm_cd', 'N/A')
    acct_method = G.nodes[node].get('acct_method', 'N/A')
    node_info = f'{node}<br>Year: {year}<br>Transformation Code: {trsfm_cd}<br>Accounting Method: {acct_method}'
    node_text.append(node_info)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    text=node_text,
    marker=dict(
        size=10,
        line_width=2,color='#CB0D1F'))

# Create the figure for plotting
fig_boa = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Network of Bank of America Acquisitions and Mergers',
                titlefont_size=16,
                showlegend=False,
                hovermode='y unified',
                xaxis=dict(title='Year of Transformation'),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                margin=dict(b=20,l=5,r=5,t=40)) # Set margins to add a title and adjust the plot size. 'b' is the bottom margin, 'l' is the left margin, 'r' is the right margin, and 't' is the top margin.
                )

#Add dark theme
fig_boa.update_layout(template='plotly_dark')

fig_boa.show()

### 3c) Network plot for Wells Fargo<a id="11"></a>

In [71]:
# Create a directed graph
G = nx.DiGraph()

# Define node positions based on the 'Year'
pos = {}
year_min = trans_wf_df['Year'].min()
year_max = trans_wf_df['Year'].max()
y_pos = 0

for _, row in trans_wf_df.iterrows():
    G.add_node(row['Predecessor_Name'], 
               year=row['Year'], 
               trsfm_cd=row['Transformation Code'], 
               acct_method=row['Accounting Method'])
    pos[row['Predecessor_Name']] = (row['Year'], y_pos)
    y_pos += 1

G.add_node('Wells Fargo')
pos['Wells Fargo'] = (year_max + 1, y_pos / 2)  # Position BoA at the end

# Add edges from predecessors to 'Chase'
for _, row in trans_wf_df.iterrows():
    G.add_edge(row['Predecessor_Name'], row['Successor_Name'])

# Create edge trace for plotly
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

# Create node trace
node_x = [pos[node][0] for node in G.nodes()]
node_y = [pos[node][1] for node in G.nodes()]
node_text = []
for node in G.nodes():
    year = G.nodes[node].get('year', 'N/A')
    trsfm_cd = G.nodes[node].get('trsfm_cd', 'N/A')
    acct_method = G.nodes[node].get('acct_method', 'N/A')
    node_info = f'{node}<br>Year: {year}<br>Transformation Code: {trsfm_cd}<br>Accounting Method: {acct_method}'
    node_text.append(node_info)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    text=node_text,
    marker=dict(
        size=10,
        line_width=2,color='#FFD408'))

# Create the figure for plotting
fig_wf = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Network of Wells Fargo Acquisitions and Mergers',
                titlefont_size=16,
                showlegend=False,
                hovermode='y unified',
                xaxis=dict(title='Year of Transformation'),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                margin=dict(b=20,l=5,r=5,t=40)) # Set margins to add a title and adjust the plot size. 'b' is the bottom margin, 'l' is the left margin, 'r' is the right margin, and 't' is the top margin.
                )

#Add dark theme
fig_wf.update_layout(template='plotly_dark')

fig_wf.show()

### 3d) Network plot for Citibank <a id="12"></a>

In [72]:
# Create a directed graph
G = nx.DiGraph()

# Define node positions based on the 'Year'
pos = {}
year_min = trans_cb_df['Year'].min()
year_max = trans_cb_df['Year'].max()
y_pos = 0

for _, row in trans_cb_df.iterrows():
    G.add_node(row['Predecessor_Name'], 
               year=row['Year'], 
               trsfm_cd=row['Transformation Code'], 
               acct_method=row['Accounting Method'])
    pos[row['Predecessor_Name']] = (row['Year'], y_pos)
    y_pos += 1

G.add_node('Citibank')
pos['Citibank'] = (year_max + 1, y_pos / 2)  # Position BoA at the end

# Add edges from predecessors to 'Chase'
for _, row in trans_cb_df.iterrows():
    G.add_edge(row['Predecessor_Name'], row['Successor_Name'])

# Create edge trace for plotly
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

# Create node trace
node_x = [pos[node][0] for node in G.nodes()]
node_y = [pos[node][1] for node in G.nodes()]
node_text = []
for node in G.nodes():
    year = G.nodes[node].get('year', 'N/A')
    trsfm_cd = G.nodes[node].get('trsfm_cd', 'N/A')
    acct_method = G.nodes[node].get('acct_method', 'N/A')
    node_info = f'{node}<br>Year: {year}<br>Transformation Code: {trsfm_cd}<br>Accounting Method: {acct_method}'
    node_text.append(node_info)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    text=node_text,
    marker=dict(
        size=10,
        line_width=2,color='#1598C3'))

# Create the figure for plotting
fig_cb = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Network of Citigroup Acquisitions and Mergers',
                titlefont_size=16,
                showlegend=False,
                hovermode='y unified',
                xaxis=dict(title='Year of Transformation'),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                margin=dict(b=20,l=5,r=5,t=40)) # Set margins to add a title and adjust the plot size. 'b' is the bottom margin, 'l' is the left margin, 'r' is the right margin, and 't' is the top margin.
                )

#Add dark theme
fig_cb.update_layout(template='plotly_dark')

fig_cb.show()